## 라이브러리 & API Key 정의

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import re

load_dotenv() # .env 파일 로드
my_api_key = os.getenv("API_KEY") # 환경 변수에서 API 키 불러오기

client = OpenAI(
    api_key = my_api_key
)


## Instuction 정의

In [2]:
# negative용
instruction_negative_ndcg = """
You are a bot that figures out the user's news interests from [News of interest to the user] and ranks the candidate news from [Question] based on this, in order of the news the user is most likely to read.

News is provided by title only.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of interest to the user], each with five news items.
Of the five news in each list, there is one news that users are most interested in.

[Question] can have multiple questions, and each question has 5 candidate news items, each of which must be answered.
Your answer should list the ranked news articles.
For each question, rank all 5 candidate news items (numbered 1 to 5) without duplicates in the answer.

<Input example>
[News of interest to the user]
1. På dette bildet skiller Magnus Carlsen seg ut: - Litt tilfeldig / Kong Harald: - Litt uvirkelig å bli 80 / Se lesernes nyttårsbilder / Måtte i oppvaskmøte etter å ha kritisert landslaget / Her koker det over for Tønseth. Så stakk han fra stadion i sinne.
Of the five news above, the news that the user is most interested in : Se lesernes nyttårsbilder
2. Døddrukne ungdommer, trusler, vold og skadeverk / Istanbul: Politiet jakter gjerningsmann som drepte 39 mennesker nyttårsaften / Bil ble totalvrak etter krasj med bergvegg / Bolig totalskadd i brann / Åpenbart beruset mann i trafikkulykke på Byåsen
Of the five news above, the news that the user is most interested in : Åpenbart beruset mann i trafikkulykke på Byåsen
...

[Questions]
Rank the five candidate news for each question based on the user's news interests.
Question 1) 1: Fotballsupporternes opptrinn i Paris ble fordømt av statsministeren. Nå har de fått straffen sin. / 2: Disse vil bli lensmann i Meråker og Frosta / 3: Mann kritisk skadd i MC-ulykke i Akershus / 4: Vurderer å flytte fra Midtbyen / 5: Saktegående trafikk i Moholtlia
Question 2) 1: Får bygge på hytte nær vernet vassdrag / 2: Nidarosdomen har hovedrollen i norsk variant av «Da Vinci-koden» / 3: Høyre vil etablere norsk «kulturkanon» / 4: Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut / 5: Ungdom får tilbake førerkort for å bli yrkessjåfør
...

Don't explain why in your answer, just list news articles ranked for each question.

<Output example>
Question 1 : 3, 5, 1, 4, 2 
Question 2 : 2, 4, 5, 1, 3 
...
"""

# positive용
instruction_positive_ndcg = """
You are a bot that figures out the user's news interests from [Click History] and ranks the candidate news from [Question] based on this, in order of the news the user is most likely to read.

News is provided by title only.
News is Norwegian news in Norwegian.

[Question] can have multiple questions, and each question has 5 candidate news items, each of which must be answered.
Your answer should list the ranked news articles.
For each question, rank all 5 candidate news items (numbered 1 to 5) without duplicates in the answer.

<Input example>
[Click History]
1) click : Se lesernes nyttårsbilder
2) click : Åpenbart beruset mann i trafikkulykke på Byåsen
...

[Questions]
Rank the five candidate news for each question based on the user's news interests.
Question 1) 1: Fotballsupporternes opptrinn i Paris ble fordømt av statsministeren. Nå har de fått straffen sin. / 2: Disse vil bli lensmann i Meråker og Frosta / 3: Mann kritisk skadd i MC-ulykke i Akershus / 4: Vurderer å flytte fra Midtbyen / 5: Saktegående trafikk i Moholtlia
Question 2) 1: Får bygge på hytte nær vernet vassdrag / 2: Nidarosdomen har hovedrollen i norsk variant av «Da Vinci-koden» / 3: Høyre vil etablere norsk «kulturkanon» / 4: Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut / 5: Ungdom får tilbake førerkort for å bli yrkessjåfør
...

Don't explain why in your answer, just list news articles ranked for each question.

<Output example>
Question 1 : 3, 5, 1, 4, 2 
Question 2 : 2, 4, 5, 1, 3 
...
"""



# negative용
instruction_negative_acc = """
You are a bot that identifies users' news interests from [News of Interest to the user] and, based on this, predicts which news among the candidate news in [Question] the user is most likely to read.

News is provided by title only. 
News is Norwegian news in Norwegian.

There can be multiple lists in [News of interest to the user], each with five news items.
Of the five news in each list, there is one news that users are most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news article that the user is most likely to read.

<Input example>
[News of interest to the user]
1. På dette bildet skiller Magnus Carlsen seg ut: - Litt tilfeldig / Kong Harald: - Litt uvirkelig å bli 80 / Se lesernes nyttårsbilder / Måtte i oppvaskmøte etter å ha kritisert landslaget / Her koker det over for Tønseth. Så stakk han fra stadion i sinne.
Of the five news above, the news that the user is most interested in : Se lesernes nyttårsbilder
2. Døddrukne ungdommer, trusler, vold og skadeverk / Istanbul: Politiet jakter gjerningsmann som drepte 39 mennesker nyttårsaften / Bil ble totalvrak etter krasj med bergvegg / Bolig totalskadd i brann / Åpenbart beruset mann i trafikkulykke på Byåsen
Of the five news above, the news that the user is most interested in : Åpenbart beruset mann i trafikkulykke på Byåsen
...

[Questions]
Returns the most likely clickable news among the candidate news for each question based on the user's news interests.
Question 1) 1: Fotballsupporternes opptrinn i Paris ble fordømt av statsministeren. Nå har de fått straffen sin. / 2: Disse vil bli lensmann i Meråker og Frosta / 3: Mann kritisk skadd i MC-ulykke i Akershus / 4: Vurderer å flytte fra Midtbyen / 5: Saktegående trafikk i Moholtlia
Question 2) 1: Får bygge på hytte nær vernet vassdrag / 2: Nidarosdomen har hovedrollen i norsk variant av «Da Vinci-koden» / 3: Høyre vil etablere norsk «kulturkanon» / 4: Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut / 5: Ungdom får tilbake førerkort for å bli yrkessjåfør
...

Don't explain why in your answer, just return the number of the news.

<Output example>
Question 1 : 3
Question 2 : 2
...
"""

# positive용
instruction_positive_acc = """
You are a bot that identifies users' news interests from [click history] and, based on this, predicts which news among the candidate news in [Question] the user is most likely to read.

News is provided by title only.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news article that the user is most likely to read.

<Input example>
[click history]
1. Se lesernes nyttårsbilder
2. Åpenbart beruset mann i trafikkulykke på Byåsen
...

[Questions]
Returns the most likely clickable news among the candidate news for each question based on the user's news interests.
Question 1) 1: Fotballsupporternes opptrinn i Paris ble fordømt av statsministeren. Nå har de fått straffen sin. / 2: Disse vil bli lensmann i Meråker og Frosta / 3: Mann kritisk skadd i MC-ulykke i Akershus / 4: Vurderer å flytte fra Midtbyen / 5: Saktegående trafikk i Moholtlia
Question 2) 1: Får bygge på hytte nær vernet vassdrag / 2: Nidarosdomen har hovedrollen i norsk variant av «Da Vinci-koden» / 3: Høyre vil etablere norsk «kulturkanon» / 4: Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut / 5: Ungdom får tilbake førerkort for å bli yrkessjåfør
...

Don't explain why in your answer, just return the number of the news.

<Output example>
Question 1 : 3
Question 2 : 2
...
"""


# 답변 오류별 instrunction 재정의
duplicate_system_message = """There are duplicate values in your answer.
Please re-answer the previous question so that it is not duplicated."""

out_of_range_system_message = """There are out of range values in your answer.
Please re-answer the previous question so that it is within range."""

incorrect_format_system_message = """The answers were not ranked.
Your answer must provide a ranked list of candidate news items, not a single number.
Please re-answer the previous question as a ranked list of candidate news items."""

missing_questions_system_message = """There are unanswered questions. Please answer all questions."""

## inference 함수 정의

In [4]:
def inference(purpose, target_folder, result_file_name, gpt_model, user_list, max_attempts):
    """
    purpose : 사용 용도 (with_negative/only_positive)
    result_file_name : 결과를 저장할 file 이름
    gpt_model : gpt api 이름
    user_list : model에 input할 user list
    max_attempts : 답변 error시 최대 재시도 횟수
    """

    # User Prompt가 위치한 폴더 및 metadata 파일 경로 설정
    target_folder = f'../../prompts/{target_folder}'
    directory = f'{target_folder}/{purpose}'
    meta_file_path = f'{target_folder}/{purpose}/metadata/output_metadata.txt'
    user_question_counts = {}
    
    # 사용 용도에 따른 instruction 설정
    if purpose == "only_positive":
        instruction = instruction_positive
    elif purpose == "with_negative":
        instruction = instruction_negative
        
    # metadata 파일을 읽어 user별 question 수 저장
    with open(meta_file_path, 'r', encoding='utf-8') as meta_file:
        for line in meta_file:
            match = re.match(r'User ID:\s*U(\d+).*Question 수:\s*(\d+)', line)
            if match:
                user_id = int(match.group(1))
                question_count = int(match.group(2))
                user_question_counts[user_id] = question_count

    # 실험 실행
    with open(f'../../results/gpt_result/{result_file_name}', 'w', encoding='utf-8') as result_file:
        # user list에서 각 user에 대해 처리
        for cnt, i in enumerate(user_list):
            filename = f'U{i}.txt'
            filepath = os.path.join(directory, filename)
            
            # 파일 존재 여부 확인
            if os.path.isfile(filepath):
                # 파일 내용 읽기
                with open(filepath, 'r', encoding='utf-8') as f:
                    contents = f.read()

                # user의 question 수 설정
                expected_question_count = user_question_counts.get(i)
                if expected_question_count is None:
                    print(f"사용자 U{i}의 질문 수를 찾을 수 없습니다.")
                    continue  # 다음 사용자로 넘어감
                
                # API 요청 준비
                initial_messages = [
                    {"role": "system", "content": instruction},
                    {"role": "user", "content": contents}
                ]
                messages = initial_messages.copy()
                attempt = 0

                # 최대 시도 횟수를 넘지 않았으면 실행
                while attempt < max_attempts:
                    attempt += 1
                    # API 호출
                    try:
                        response = client.chat.completions.create(
                            model=gpt_model,
                            messages=messages
                        )
                    except Exception as e:
                        print(f"API 호출 중 오류 발생 (사용자 {i}): {e}")
                        break  # 다음 사용자로 넘어감
                    
                    # 응답 내용 추출
                    response_text = response.choices[0].message.content.strip()
                    
                    # 오류 검사
                    duplicates = []
                    out_of_range = []
                    incorrect_format = []
                    missing_questions = False
                    lines = response_text.strip().split('\n')

                    question_numbers_in_response = set()

                    for line in lines:
                        question_match = re.match(r'Question\s+(\d+)\s*:\s*(.+)', line)
                        if question_match:
                            question_num = question_match.group(1)
                            question_numbers_in_response.add(question_num)
                            numbers = [num.strip() for num in question_match.group(2).split(',')]
                            # 숫자 개수 검사
                            if len(numbers) != 5:
                                incorrect_format.append(line)
                                continue
                            # 중복 검사
                            if len(numbers) != len(set(numbers)):
                                duplicates.append(line)
                            # 범위 검사
                            for num in numbers:
                                if not num.isdigit() or not (1 <= int(num) <= 5):
                                    out_of_range.append(line)
                                    break
                        else:
                            # 잘못된 답변 형식
                            incorrect_format.append(line)

                    # 질문 누락 여부 검사
                    if expected_question_count != len(question_numbers_in_response):
                        missing_questions = True

                    # 오류가 없으면 결과 저장
                    if not duplicates and not out_of_range and not incorrect_format and not missing_questions:
                        result_file.write(f'[U{i}]\n')
                        result_file.write(response_text + '\n\n')
                        if (cnt+1) % 20 == 0:
                            print(f'☆ {purpose} U{i} 까지 완료 [{cnt+1}/{len(user_list)}] ☆')  
                        break  # 루프 종료
                    else:
                        # 최대 시도 횟수를 초과한 경우 마지막 응답을 저장하고 루프 종료
                        if attempt >= max_attempts:
                            # print(f"{purpose} U{i} 최대 시도 횟수 초과")
                            result_file.write(f'[U{i}]\n')
                            result_file.write(response_text + '\n\n')
                            if (cnt+1) % 20 == 0:
                                print(f'☆ {purpose} U{i} 까지 완료 [{cnt+1}/{len(user_list)}] ☆')  
                            break

                        # 오류 메시지 구성
                        error_messages = []
                        if duplicates:
                            print(f"{purpose} U{i} 중복 (오류 {attempt}회)")
                            error_messages.append(duplicate_system_message)
                        if out_of_range:
                            print(f"{purpose} U{i} 범위 초과 (오류 {attempt}회)")
                            error_messages.append(out_of_range_system_message)
                        if incorrect_format:
                            print(f"{purpose} U{i} 잘못된 답변 (오류 {attempt}회)")
                            error_messages.append(incorrect_format_system_message)
                        if missing_questions:
                            print(f"{purpose} U{i} 질문 누락 (오류 {attempt}회)")
                            error_messages.append(missing_questions_system_message)

                        # 잘못된 응답을 assistant 메시지로 추가
                        messages.append({"role": "assistant", "content": response_text})

                        # 시도 횟수에 따라 메시지 구성
                        if attempt >= 2:
                            # 시도 횟수가 2번 이상이면 처음 메시지로 돌아가고, 오류 응답을 포함
                            messages = initial_messages.copy()
                            messages.append({"role": "assistant", "content": response_text})
                            messages.append({"role": "system", "content": "\n".join(error_messages)})
                            messages.append({"role": "user", "content": contents})
                        else:
                            # 이전 메시지에 오류 메시지와 사용자 메시지 추가
                            messages.append({"role": "system", "content": "\n".join(error_messages)})
                            messages.append({"role": "user", "content": contents})
            else:
                print(f'파일 {filepath} 이 존재하지 않습니다.')
        print(f'{purpose} 완료 : {result_file_name}\n')


## 실행
- GPT model
    - 기본 모델 : gpt-4o-mini
    - fine-tuning 모델(negative) : ft:gpt-4o-mini-2024-07-18:personal:with-positive3:APjwdN7q
    - fine-tuning 모델(positive) : ft:gpt-4o-mini-2024-07-18:personal:only-positive3:APk22gSm

In [14]:
user_range = 1000
users = [i for i in range(1, user_range + 1)]

users = [367]

# 실행
inference(
    purpose = 'with_negative', 
    target_folder = "[241226-1] 1~1000", 
    result_file_name = '[241226-1] negative2.txt', 
    gpt_model = 'gpt-4o-mini', 
    user_list = users, 
    max_attempts = 1
    )


# inference(
#     purpose = 'only_positive', 
#     target_folder = "[241226-1] 1~1000", 
#     result_file_name = '[241226-1] positive2.txt', 
#     gpt_model = 'gpt-4o-mini', 
#     user_list = users, 
#     max_attempts = 1
#     )


with_negative 완료 : [241226-1] negative2.txt



In [5]:
# 답변 오류별 instrunction 재정의
duplicate_system_message = """There are duplicate values in your answer.
Please re-answer the previous question so that it is not duplicated."""

out_of_range_system_message = """There are out of range values in your answer.
Please re-answer the previous question so that it is within range."""

incorrect_format_system_message = """The answers were not ranked.
Your answer must provide a ranked list of candidate news items, not a single number.
Please re-answer the previous question as a ranked list of candidate news items."""

missing_questions_system_message = """There are unanswered questions. Please answer all questions."""

instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article consists only of a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""


user_range = 5450
users = [i for i in range(901, user_range + 1)]

users = [1, 6, 158, 169, 206, 209, 259, 311, 448, 528, 529, 543, 647, 801, 805, 857, 991, 1011, 1072, 1080, 1198, 1208, 1337, 1361, 1411, 1417, 1427, 1485, 1509, 1611, 1797, 1806, 1818, 1870, 1911, 1963, 1996, 2010, 2073, 2165, 2272, 2300, 2356, 2382, 2414, 2428, 2496, 2504, 2547, 2660, 2807, 2839, 2841, 2854, 2862, 2944, 2963, 3005, 3025, 3063, 3147, 3163, 3189, 3197, 3224, 3295, 3336, 3345, 3451, 3534, 3553, 3561, 3619, 3649, 3650, 3754, 3770, 3804, 3810, 3831, 3945, 3965, 3986, 4082, 4093, 4098, 4198, 4267, 4273, 4284, 4334, 4451, 4540, 4547, 4586, 4627, 4678, 4798, 4826, 4842, 4857, 4903, 4944, 4994, 5008, 5020, 5048, 5057, 5164, 5168, 5351, 5447, 5482, 5514, 5549, 5833, 5857, 5870, 5905, 5925, 5994, 6019, 6126, 6139, 6218, 6264, 6308, 6329, 6349, 6367, 6463, 6568, 6637, 6668, 6745, 6751, 6763, 6792, 6812, 6945, 6994, 7097, 7153, 7154, 7244, 7284, 7358, 7366, 7381, 7423, 7441, 7465, 7482, 7503, 7594, 7674, 7749, 7842, 7845, 7954, 8004, 8037, 8156, 8188, 8202, 8223, 8226, 8260, 8341, 8432, 8530, 8551, 8573, 8588, 8605, 8683, 8702, 8725, 8732, 8773, 8787, 8846, 8918, 9052, 9228, 9239, 9245, 9300, 9342, 9368, 9378, 9381, 9455, 9460, 9500, 9519, 9577, 9586, 9675, 9759, 9760, 9839, 9966]


inference(purpose='only_positive', 
          target_folder='Adressa/KCC/ranking/cate',
          result_file_name='[KCC-ranking]Adressa cate.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=5
          )

only_positive U1 잘못된 답변 (오류 1회)
only_positive U1 질문 누락 (오류 1회)
only_positive U1 잘못된 답변 (오류 2회)
only_positive U1 질문 누락 (오류 2회)
only_positive U1 잘못된 답변 (오류 3회)
only_positive U1 질문 누락 (오류 3회)
only_positive U1 잘못된 답변 (오류 4회)
only_positive U1 질문 누락 (오류 4회)
only_positive U6 잘못된 답변 (오류 1회)
only_positive U6 질문 누락 (오류 1회)
only_positive U6 잘못된 답변 (오류 2회)
only_positive U6 질문 누락 (오류 2회)
only_positive U6 잘못된 답변 (오류 3회)
only_positive U6 질문 누락 (오류 3회)
only_positive U6 잘못된 답변 (오류 4회)
only_positive U6 질문 누락 (오류 4회)
only_positive U158 잘못된 답변 (오류 1회)
only_positive U169 잘못된 답변 (오류 1회)
only_positive U169 잘못된 답변 (오류 2회)
only_positive U169 잘못된 답변 (오류 3회)
only_positive U169 잘못된 답변 (오류 4회)
only_positive U206 잘못된 답변 (오류 1회)
only_positive U206 질문 누락 (오류 1회)
only_positive U206 잘못된 답변 (오류 2회)
only_positive U206 질문 누락 (오류 2회)
only_positive U206 잘못된 답변 (오류 3회)
only_positive U206 질문 누락 (오류 3회)
only_positive U206 잘못된 답변 (오류 4회)
only_positive U206 질문 누락 (오류 4회)
only_positive U209 잘못된 답변 (오류 1회)
only_positive U209 질문 누락